<a href="https://colab.research.google.com/github/GuilhermeSano/APIgemini_sheets/blob/main/gemini%2Bapisheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import google.generativeai as genai
import google.auth
from google.colab import auth
from googleapiclient.discovery import build
import hashlib

auth.authenticate_user()
#É necessário gerar uma API_KEY no site https://aistudio.google.com/app/apikey
GOOGLE_API_KEY = "SUA_APIKEY_AQUI"
genai.configure(api_key=GOOGLE_API_KEY)
#É necessário ligar o API do google sheets no site https://console.cloud.google.com, a planilha abaixo é de testes apenas.
SPREADSHEET_ID = '1T9seWy7AJeXKg0Jhg2pDSyeI0QuVTA4b86VaYnn_a0Y'
MODEL = "models/embedding-001"

#Cache de embeddings
cache_embeddings = {}

#Função para gerar embeddings com cache
def gerar_embedding(texto):
    chave_cache = hashlib.md5(texto.encode()).hexdigest()
    if chave_cache in cache_embeddings:
        return cache_embeddings[chave_cache]
    else:
        embedding = genai.embed_content(
            model=MODEL,
            content=texto,
            task_type="RETRIEVAL_DOCUMENT"
        )["embedding"]
        cache_embeddings[chave_cache] = embedding
        return embedding

#Função para buscar dados da planilha
def get_data_from_sheet(sheet_name):
    creds, _ = google.auth.default()
    service = build('sheets', 'v4', credentials=creds)
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=sheet_name).execute()
    values = result.get('values', [])

    if values:  #Verifica se values não está vazio
        if len(values) > 1:
            if values[0] and isinstance(values[0][0], str):
                return pd.DataFrame(values[1:], columns=values[0])
            else:
                return pd.DataFrame(values)
        elif len(values) == 1 and len(values[0]) > 0:
            return pd.DataFrame(data=[values[0]])
        else:
            return pd.DataFrame()
    else:
        return pd.DataFrame()  #Retorna um DataFrame vazio se values estiver vazio

#Função para buscar a linha mais relevante
def buscar_documento(consulta, df_all):
    embedding_consulta = gerar_embedding(consulta)
    melhor_pontuacao = float("-inf")
    melhor_linha = None
    for sheet_name, df in df_all.items():
        for i, linha in df.iterrows():
            if linha.dropna().empty:  #Verifica se a linha está vazia
                continue
            conteudo_linha = ' '.join(str(x) for x in linha.values)
            embedding_linha = gerar_embedding(conteudo_linha)
            pontuacao = np.dot(embedding_linha, embedding_consulta)
            if linha is not None:  #Verifica se a linha não é None
                if pontuacao > melhor_pontuacao:
                    melhor_pontuacao = pontuacao
                    melhor_linha = linha
    if melhor_linha is not None:
        return melhor_linha["Conteudo"]
    else:
        return None

#Carregar dados da planilha
creds, _ = google.auth.default()
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.get(spreadsheetId=SPREADSHEET_ID).execute()
abas = result.get('sheets', [])
df_all = {aba['properties']['title']: get_data_from_sheet(aba['properties']['title']) for aba in abas if get_data_from_sheet(aba['properties']['title']) is not None}

#Loop principal
while True:
    consulta = input('Digite sua consulta (ou "sair" para encerrar): ')
    if consulta.lower() == "sair":
        break
    linha = buscar_documento(consulta, df_all)
    if linha is not None:
        print(linha)
    else:
        print("Nenhum resultado encontrado.")

Digite sua consulta (ou "sair" para encerrar): perfumes
Chanel Nº 5, Dior J'adore, Yves Saint Laurent Black Opium, Gucci Bloom, Lancôme La Vie Est Belle, Carolina Herrera Good Girl, Paco Rabanne 1 Million, Giorgio Armani Acqua di Gio, Dolce & Gabbana Light Blue, Thierry Mugler Angel.
Digite sua consulta (ou "sair" para encerrar): sair
